In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install --upgrade --force-reinstall --no-deps albumentations

  Using cached https://files.pythonhosted.org/packages/e7/27/2fa0ec5e0c04c410cbb54dd79910afa884409440653aa4688654e6497e2a/albumentations-1.0.2-py3-none-any.whl
  Found existing installation: albumentations 1.0.2
    Uninstalling albumentations-1.0.2:
      Successfully uninstalled albumentations-1.0.2


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms.functional as TF

from torch.utils.data import DataLoader
import torch.nn.functional as F

import torch.optim as optim

import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import cv2
from os import path
import torch
from skimage.io import imread
from torch.utils import data

# **Unet Architecture**

In [ ]:
class UNet(nn.Module):
    def __init__(
        self,
        in_channels=3,
        n_classes=1,
        depth=4,
        wf=6,
        padding=False,
        batch_norm=False,
        up_mode='upconv',
    ):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path) - 1:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

# Train and validation functions

In [ ]:
from tqdm import tqdm

In [ ]:
def training_function(loader, model, optimizer, loss_fn):
  loop = tqdm(loader)
  trainingLossSum=0
  for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)     
      targets = targets.float().to(device=DEVICE)
      predictions = model(data)#.permute(0, 1, 3,2)
      predictions= F.pad(input=predictions, pad=(0, 10, 8, 2), mode='constant', value=0)
      train_loss = loss_fn(predictions, targets)
      trainingLossSum+=train_loss                  
      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()
      # update tqdm loop
      loop.set_postfix(loss=train_loss.item())
  return trainingLossSum/len(loader)

In [ ]:
def validating_function(loader,model,optimizer,loss_fn):
  loop = tqdm(loader)
  valLossSum=0
  for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)
      targets = targets.float().to(device=DEVICE)
      with torch.no_grad(): # <1> dont build graph on validation
              predictions = model(data)
              predictions = F.pad(input=predictions, pad=(0, 10, 8, 2), mode='constant', value=0)
              val_loss = loss_fn(predictions, targets)
              valLossSum+=  val_loss   
              assert val_loss.requires_grad == False # <2> checks that our output requires_grad
              # args are forced to False inside this block
               
      optimizer.zero_grad()
      optimizer.step()
       # update tqdm loop
      loop.set_postfix(loss=val_loss.item())
  return valLossSum/len(loader)

Preparing data

In [ ]:
from torchvision import transforms
to_tensor = transforms.ToTensor()

In [ ]:
def replaceInputName(mask_path,diseaseType):
  result = mask_path.index('IDRiD')
  newString=''

  if diseaseType==1:  #for Hard exudates
    newString= '_EX'
    
  elif diseaseType==2: #for hemorrages
    newString= '_HE'
  elif diseaseType==3:  
    newString= '_SE'
  else:
    newString= '_MA'

  toReplace= mask_path[result:result+8]
  mask_path=mask_path.replace(toReplace, toReplace+newString)
  mask_path=mask_path.replace('images','groundtruths')
  return mask_path

class CarvanaDataset(Dataset):
   
    def __init__(self, diseaseType,image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.diseaseType= diseaseType

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = replaceInputName(img_path,diseaseType)


        image= cv2.imread(img_path)
        image = np.array(image, dtype=np.float32)
        
        mask = cv2.imread(mask_path)
        mask = mask[:,:,2]
        mask[mask == 255.0] = 1.0


        #DELETE
        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            #check to do augmentation IF and only IF the mask has 1 on it
            image = augmentations["image"]
            mask = augmentations["mask"]
            #check to do augmentation IF and only IF the mask has 1 on it
        #DELETE
        return image, mask

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps albumentations

In [ ]:
LEARNING_RATE = 5e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Learning rate is ", LEARNING_RATE)
import torch.optim as optim
device=DEVICE
filename=''
BATCH_SIZE =4
NUM_EPOCHS = 500
NUM_WORKERS = 1
UnetDepth=5
model = UNet(n_classes=1, padding=True,batch_norm=True, up_mode='upsample',depth=UnetDepth).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

PIN_MEMORY = True
LOAD_MODEL = False

Learning rate is  5e-05


In [ ]:
lossFunction=4

In [ ]:
LEARNING_RATE=0
if lossFunction==1:
    LEARNING_RATE = 5e-5
    ALPHA = 0.5
    BETA = 0.5
    # https://arxiv.org/abs/1706.05721
    #optimise segmentation on imbalanced medical datasets
    # in the case of α=β=0.5 the Tversky index simplifies to be the same as the Dice coefficient, which is also equal to the F1 score
    #modify beta increasing, so FN are punished harder
    class TverskyLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(TverskyLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
            
            #comment out if your model contains a sigmoid or equivalent activation layer
            inputs = F.sigmoid(inputs)       
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            TP = (inputs * targets).sum()    
            FP = ((1-targets) * inputs).sum()
            FN = (targets * (1-inputs)).sum()
          
            Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
            
            return 1 - Tversky
    loss_fn = TverskyLoss().to(DEVICE)
    filename="TverskyLossModel"+str(ALPHA)+str(BETA)+"UnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
    print("TverskyLoss selected")
    print(filename)
elif lossFunction==2:
      LEARNING_RATE = 5e-5
      #combo loss :  https://arxiv.org/abs/1805.02798
      #Combo loss is a combination of Dice Loss and a modified Cross-Entropy function that,
      # like Tversky loss, has additional constants which penalise either false positives or false negatives more respectively.
      ##PyTorch
      ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
      CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss
      e=1e-7
      class ComboLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(ComboLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            intersection = (inputs * targets).sum()    
            dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
            
            inputs = torch.clamp(inputs, e, 1.0 - e)       
            out = - (ALPHA * ((targets * torch.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * torch.log(1.0 - inputs))))
            weighted_ce = out.mean(-1)
            combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)
            
            return combo
      loss_fn = ComboLoss().to(DEVICE)
      filename="ComboLossModel"+str(alpha)+str(1-alpha)+"UnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
      print("Combo loss selected")
      print(filename)
elif lossFunction == 3:
      LEARNING_RATE = 5e-5
      #Focal Loss
      #for combatting extremely imbalanced datasets https://arxiv.org/abs/1708.02002

      ALPHA = 0.8 #DONT MODIFY
      GAMMA = 2  #DONT MODIFY

      class FocalLoss(nn.Module):
          def __init__(self, weight=None, size_average=True):
              super(FocalLoss, self).__init__()

          def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
              
              #comment out if your model contains a sigmoid or equivalent activation layer
              inputs = F.sigmoid(inputs)       
              
              #flatten label and prediction tensors
              inputs = inputs.view(-1)
              targets = targets.view(-1)
              
              #first compute binary cross-entropy 
              BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
              BCE_EXP = torch.exp(-BCE)
              focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                            
              return focal_loss
      loss_fn = FocalLoss().to(DEVICE)
      filename="FocalLossModelA"+str(ALPHA)+"Unet"+"DEPTH"+str(UnetDepth)+".pth.tar"
      print("Focal loss selected")
      print(filename)
elif lossFunction==4:
    #focal tversky loss
    #includes the gamma correction from focal loss
    ALPHA = 0.8 #penalizes FP 
    BETA = 0.2 #penalizes FN
    GAMMA = 2
    LEARNING_RATE = 5e-5
    alpha=ALPHA
    beta=BETA
    class FocalTverskyLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(FocalTverskyLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA, gamma=GAMMA):
            
            #comment out if your model contains a sigmoid or equivalent activation layer
            inputs = F.sigmoid(inputs)       
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            TP = (inputs * targets).sum()    
            FP = ((1-targets) * inputs).sum()
            FN = (targets * (1-inputs)).sum()
            
            Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
            FocalTversky = (1 - Tversky)**gamma
                          
            return FocalTversky
    loss_fn = FocalTverskyLoss().to(DEVICE)
    filename="FocalTverskyLossModel"+str(alpha)+str(beta)+"gamma"+str(GAMMA)+"Unet"+str(UnetDepth)+".pth.tar"
    print("tversky loss selected"+str(alpha)+str(beta)+str(GAMMA))
    print(filename)

else:
    loss_fn = nn.BCEWithLogitsLoss()
    filename="BCEWithLogitsLossLossModelSuperAugmentedUnet"+"DEPTH"+str(UnetDepth)+".pth.tar"
    print('bcewithlogits')
    print(filename)



In [ ]:
# #Hard Exudate
#diseaseType=1

#HEmo
diseaseType=2

# #soft exudates
#diseaseType=3

# #micro an
#diseaseType=4

root = '/content/drive/MyDrive/Colab Notebooks/'
folder=''
diseaseFolder=''

if diseaseType==1:  #for Hard exudates
  %cd /content/drive/My Drive/Colab Notebooks/hard exudates new unet
  diseaseFolder='hard exudates new unet'
  
elif diseaseType==2: #for hemorrages
  diseaseFolder='hemorrages full new unet'
  %cd /content/drive/My Drive/Colab Notebooks/hemorrages full new unet

elif diseaseType==3: #for soft exudates
  diseaseFolder='soft exudates new unet'
  %cd /content/drive/My Drive/Colab Notebooks/soft exudates new unet

else: #microaneurysms
  diseaseFolder='microaneurysms new unet sliced'
  %cd /content/drive/My Drive/Colab Notebooks/microaneurysms new unet sliced

folder='/content/drive/My Drive/Colab Notebooks/'+diseaseFolder
TRAIN_IMG_DIR = root+diseaseFolder+"/images/training"
TRAIN_MASK_DIR = root+diseaseFolder+"/groundtruths/training"
VAL_IMG_DIR = root+diseaseFolder+"/images/test"
VAL_MASK_DIR = root+diseaseFolder+"/groundtruths/test"

filename = "LR"+str(LEARNING_RATE)+(diseaseFolder+filename).replace(" ", "")
print(filename)

In [ ]:
from torch.utils.data import DataLoader

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset(
                diseaseType,
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform

    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CarvanaDataset(
                diseaseType,
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform

    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

Training

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
train_transform = A.Compose(
    [
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=5, border_mode=0, p=0.5),
        A.GridDistortion(p=0.3),
        #A.ChannelShuffle(),
        A.OneOf([
                          A.MotionBlur(p=0.1),
                          A.OpticalDistortion(p=0.1)
                          ,A.GaussNoise(p=0.1)
                          ], p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)
val_transforms = A.Compose(
    [
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

In [ ]:
import torch.optim as optim
train_loader, val_loader = get_loaders(
    TRAIN_IMG_DIR,
    TRAIN_MASK_DIR,
    VAL_IMG_DIR,
    VAL_MASK_DIR,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)

In [ ]:
pip install torchinfo

In [ ]:
from torchinfo import summary

batch_size = 4
summary(model, input_size=(batch_size, 3, 474, 714))

In [ ]:
def save_checkpoint(state, filename):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [ ]:
optim = torch.optim.Adam(model.parameters())
import torch.nn as nn
bestLoss=1000
#loss function
#loss_fn= nn.BCEWithLogitsLoss()
trainingLoss=[]
validationLoss=[]

n=0

for epoch in range(NUM_EPOCHS):
  n+=1
  print('epoch: ',n)
  tloss=training_function(train_loader,model,optim,loss_fn)
  print('tloss='+str(tloss))
  trainingLoss.append(tloss)
  vloss=validating_function(val_loader,model,optim,loss_fn)
  validationLoss.append(vloss)
  print('vloss: '+ str(vloss))
  if bestLoss> vloss:
    bestLoss=vloss
    checkpoint = {
      "state_dict": model.state_dict(),
      "optimizer":optimizer.state_dict(),
    }
    save_checkpoint(checkpoint,filename)

In [ ]:
def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds=model(x)
            print(torch.unique(preds))
            preds = torch.sigmoid(preds)
            preds[preds>0.5] = 1.0
            preds[preds<=0.5] = 0.0
            pre = 255.0 * np.array(preds.permute(0, 2, 3, 1).to("cpu").detach())
            print(pre.shape)
            print("Now starting")
            for i in range(BATCH_SIZE):
              print(pre[i].shape)
              abc= "/content/drive/My Drive/hemorrages08/pred_"+str(idx)+" value "+str(i)+".jpg"
              writeStatus = cv2.imwrite(abc, pre[i])
              if writeStatus is True:
                  print("image written")
              else:
                  print("problem") # or raise exception, handle problem, etc.

In [ ]:
#load_checkpoint(torch.load("TverskyLossModel0.50.5UnetEbaneoDEPTH5.pth.tar"), model)
save_predictions_as_imgs(
        val_loader, model, folder="/content/drive/MyDrive/Colab Notebooks/saved_images", device=DEVICE
    )

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, auc, precision_recall_curve
import matplotlib.pyplot as plt

In [ ]:
def evaluate_precission_recall(
    loader, model, device="cuda"
):
    
    model.eval()
    predictions_array=[]
    targets_array=[]
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            targets = y.cpu().data.numpy()
            targets = torch.from_numpy(targets)
            targets = targets.float().unsqueeze(1).to(device=DEVICE)
            targets = targets.cpu().data.numpy()
            targets= targets.astype(np.float16)
            preds=model(x)
            #preds= F.pad(input=preds, pad=(0, 1, 0, 1), mode='constant', value=0) # 4 layers
            #preds= F.pad(input=preds, pad=(0, 10, 8, 2), mode='constant', value=0) # resnet 50
            preds= F.pad(input=preds, pad=(0, 10, 9, 1), mode='constant', value=0) # 5 layers
            #print('preds padd ',preds.shape)
            preds = torch.sigmoid(preds)
            preds =preds.cpu().data.numpy()
            preds = preds.astype(np.float16)

            
            predictions_array=np.concatenate((predictions_array, preds.flatten()), axis=None)
            targets_array=np.concatenate((targets_array, targets.flatten()), axis=None)


    precision, recall, th = precision_recall_curve(targets_array,predictions_array)
    return precision, recall, th

In [ ]:
# load_checkpoint(torch.load("/content/drive/MyDrive/Colab Notebooks/hemorrages full new unet/LR5e-05hemorragesfullnewunetFocalTverskyLossModel0.20.8gamma2resnetEbaneo5.pth.tar"), model)
a,b,c = evaluate_precission_recall(val_loader,model)

In [ ]:
precision =a
recall =b

In [ ]:
plt.figure()
plt.plot(recall,precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Microaneurysm')
plt.savefig('ma_focal_AUPRC_08_02_2', dpi=300, bbox_inches='tight')
plt.show()
print(f'AUC Precision Recall {str}: {auc(recall, precision)}')

In [ ]:
x = torch.randn(4, 857, 569, 3)
x.size()
#x=F.pad(input=x, pad=(1, 0, 1, 0), mode='constant', value=0)
x = x.permute(0,3, 2,1)
x.shape

In [ ]:
#load_checkpoint(torch.load("TverskyLossModel0.50.5UnetEbaneoDEPTH5.pth.tar"), model)
for batch_idx, (data, targets) in enumerate(loop):

    data = data.to(device=DEVICE)
    targets = targets.float().to(device=DEVICE)
    predictions = model(data)#.permute(0, 1, 3,2)
    preds = torch.sigmoid(predictions)
    preds[preds>0.5] = 1.0
    preds[preds<=0.5] = 0.0
    #groundtruth

    precision, recall, thresholds = precision_recall_curve(targets, preds)